# Deep Neural Networks Analysis of Audiobooks

## Data

### Data Preprocessing

In [1]:
#Import relevant libraries
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [2]:
#Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

### Balancing the data

In [3]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))
# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0
# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []
# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

### Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [5]:
# When the data was collected it might be arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the data into train, validation, and test

In [6]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count : validation_samples_count + train_samples_count]
validation_targets = shuffled_targets[train_samples_count : validation_samples_count + train_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[validation_samples_count + train_samples_count:]
test_targets = shuffled_targets[validation_samples_count + train_samples_count:]

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)


1790.0 3579 0.5001397038278849
231.0 447 0.5167785234899329
216.0 448 0.48214285714285715


### Saving the three data sets in .npz file

In [7]:
# Save the three datasets in *.npz.
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

## Create Machine Learning Algorithm

In [8]:
#this is not necessary since this is just 1 notebook. 
#But in case we run this step, this might be useful. Also, so that we know the libraries we will use here
import numpy as np
import tensorflow as tf

### Loading data

In [9]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float64)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int64)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int64)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int64)

### Modelling
Outline, optimizers, loss, early stopping and training

In [10]:
# Set the input and output sizes
input_size = 10 #rating 1-10
output_size = 2 #1-will buy 0 will not buy again/convert
# Use same hidden layer size for both hidden layers
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(hidden_layer_size, activation = 'softmax')    
])

### Choose the optimizer and the loss function
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
### Training
# set the batch size
batch_size = 100
# set a maximum number of training epochs
max_epochs = 100
# fit the model
model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping], # early stopping
          validation_data = (validation_inputs, validation_targets),
          verbose = 2   
)


Epoch 1/100
36/36 - 2s - 67ms/step - accuracy: 0.4498 - loss: 3.2940 - val_accuracy: 0.6868 - val_loss: 2.3546
Epoch 2/100
36/36 - 0s - 9ms/step - accuracy: 0.6935 - loss: 1.3093 - val_accuracy: 0.7248 - val_loss: 0.6293
Epoch 3/100
36/36 - 0s - 13ms/step - accuracy: 0.7368 - loss: 0.5472 - val_accuracy: 0.7852 - val_loss: 0.4799
Epoch 4/100
36/36 - 0s - 8ms/step - accuracy: 0.7703 - loss: 0.4735 - val_accuracy: 0.7897 - val_loss: 0.4302
Epoch 5/100
36/36 - 0s - 11ms/step - accuracy: 0.7762 - loss: 0.4377 - val_accuracy: 0.7830 - val_loss: 0.4012
Epoch 6/100
36/36 - 0s - 8ms/step - accuracy: 0.7770 - loss: 0.4151 - val_accuracy: 0.8098 - val_loss: 0.3832
Epoch 7/100
36/36 - 0s - 8ms/step - accuracy: 0.7935 - loss: 0.3982 - val_accuracy: 0.8389 - val_loss: 0.3688
Epoch 8/100
36/36 - 0s - 8ms/step - accuracy: 0.7927 - loss: 0.3887 - val_accuracy: 0.8233 - val_loss: 0.3597
Epoch 9/100
36/36 - 0s - 8ms/step - accuracy: 0.7997 - loss: 0.3789 - val_accuracy: 0.8255 - val_loss: 0.3558
Epoch 1

## Testing the Model

In [11]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)
print('nTest Loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8151 - loss: 0.3323 
nTest Loss: 0.33. Test accuracy: 80.36%
